In [5]:
%%capture
!pip install ipywidgets --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer 

# Define the model repo
model_name = "DeepPavlov/rubert-base-cased" 


# Download pytorch model
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print('model downloaded')

# Transform input tokens 
inputs = tokenizer("Hello world!", return_tensors="pt").to('cuda')

# Model apply
model.to('cuda')
model.eval()
outputs = model(**inputs)
outputs

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model downloaded


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0699, -0.0093, -0.2027,  ..., -0.0275,  0.0105, -0.2243],
         [ 0.1231, -0.3886, -0.1811,  ..., -0.1576, -0.2227, -0.7154],
         [ 0.1322, -0.2026, -0.1001,  ...,  0.0254, -0.0627, -0.7906],
         [-0.0746, -0.0255,  0.6633,  ..., -0.1301, -0.2033, -0.4783],
         [ 0.3048,  0.3046, -0.2457,  ..., -0.4703, -0.2132,  0.4014]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 2.1306e-01, -5.7159e-02,  1.9231e-01,  1.0838e-01, -1.3495e-01,
          3.2246e-01,  1.4012e-01,  1.1668e-01, -2.2484e-01,  2.4496e-01,
         -7.5188e-02, -1.7386e-01, -2.5435e-01, -1.5678e-01,  9.6359e-02,
         -2.6535e-01,  7.8951e-01,  9.4643e-02,  1.0328e-01, -1.8292e-01,
         -9.9998e-01, -6.9320e-02, -2.4173e-01, -1.1112e-01, -2.8712e-01,
          8.0301e-02, -1.3064e-01,  1.0944e-01,  1.0707e-01, -1.2861e-01,
          3.6611e-02, -9.9989e-01,  5.3055e-01,  6.4786e-01, 

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)

(312,)


In [12]:
from transformers import BertForMaskedLM,BertTokenizer, pipeline
model=BertForMaskedLM.from_pretrained('sberbank-ai/ruBert-base')
tokenizer=BertTokenizer.from_pretrained('sberbank-ai/ruBert-base', do_lower_case=False)
unmasker = pipeline('fill-mask', model=model,tokenizer=tokenizer)

[i['sequence'] for i in unmasker("Python -  это самый [MASK] язык программирования.")]

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


['Python - это самый популярный язык программирования.',
 'Python - это самый современный язык программирования.',
 'Python - это самый сложный язык программирования.',
 'Python - это самый простой язык программирования.',
 'Python - это самый распространённый язык программирования.']

In [2]:
from transformers import pipeline

task = 'feature-extraction' 
generator = pipeline(task, model="sberbank-ai/ruT5-base")
generator(
    "Текст: С мая 2021 в России повысят налоги. Тема: "
)